# Initializing and functions

In [1]:
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import sif_parser
import scipy
from scipy.signal import savgol_filter
import lmfit
from lmfit import Model
import datetime
import pandas as pd
import csv
import cv2

scl=0.45
#############################################################################
SMALL_SIZE = 30*scl
MEDIUM_SIZE = 33*scl
BIGGER_SIZE = 35*scl
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
# plt.rcParams["font.family"] = "Times New Roman"
# plt.rcParams['text.usetex'] = True
#############################################################################
%matplotlib tk
def logistic(x, a, b, c, d):
        return a / (1. + np.exp(-c * (x - d))) + b
def square(x, a, b, c):
    return a*x**2 + b*x + c

In [2]:
import numpy as np
from pyhank import qdht, HankelTransform
import matplotlib.pyplot as plt

def smoothAvg(data, box_pts):
    y = data[:,1]
    y_smooth = smoothAvgAux(y, box_pts)
    if box_pts == 0:
        return np.array([data[:,0], y_smooth]).transpose()
    else:
        return np.array([data[:-box_pts,0], y_smooth[:-box_pts]]).transpose()

def smoothAvgAux(y, box_pts):
    if box_pts == 0:
        return y
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

def lsf2psf(lsf, x, control):
    r = np.linspace(start=0.0, stop=np.max(np.abs(x)), num=control['nr'])
    lsf_mat = np.tile(np.abs(lsf).transpose(), [len(r), 1])
    X, R = np.meshgrid(x, r)
    dx = np.abs(x[2] - x[1])

    integrand = - 1 / np.pi * lsf_mat * R / ( np.abs(X) * np.sqrt(X**2 - R**2 + 1e-20)) * dx
    integrand = np.nan_to_num(integrand, nan=0) * (np.abs(X) > R)
    integral = np.sum(integrand, axis=1)
    psf = np.abs(np.diff(integral)[control['spare']:-control['spare']])

    psf = smoothAvgAux(psf, control['blurKernelSizePSF'])[control['blurKernelSizePSF']:]
    return psf

def mtf2dqe(psf, mtf, control):
    absorption = 0.08
    g1 = absorption
    g2 = np.sum(np.abs(psf))
    g3 = 0.5
    dqe = g1 / (1 + 1 / (g2 * g3 * np.abs(mtf / 100)**2))
    dqe = 100 * smoothAvgAux(dqe, control['blurKernelSizeDQE'])[control['blurKernelSizeDQE']:]
    return dqe

def psf2mtf(r, psf, control):
    """Compute the MTF of a PSF with cylindrical symmetric using Hankel transform"""
    H = HankelTransform(order=0, radial_grid=r)

    psf_high_res = np.zeros_like(r)
    psf_high_res[:len(psf)] = psf

    ht = np.abs(H.qdht(psf_high_res))
    ht /= (np.max(ht) + 1e-12)
    smoothMTF = smoothAvg(np.array([H.v, ht]).transpose(), control['blurKernelSizeMTF'])
    smoothMTF = smoothMTF[(control['blurKernelSizeMTF'] - 0):, :]
    smoothMTF[:, 0] -= smoothMTF[0, 0]
    smoothMTF[:, 1] /= (smoothMTF[0, 1] + 1e-20)
    smoothMTF[:, 1] *= 100
    return smoothMTF

def compute_image_characteristics(y, control):
    r = control['CCD_pixel_size'] * np.arange(len(y))
    esf = smoothAvgAux(y, control['blurKernelSize'])
    if control['blurKernelSize'] > 0:
        esf = esf[control['blurKernelSize']: -control['blurKernelSize']]
    lsf = np.abs(smoothAvgAux(np.diff(esf), control['blurKernelSize']))

    data = np.array([r,y]).transpose()
    data = smoothAvg(data, control['blurKernelSize'])
    data[:,0] -= data[0,0]

    x = control['CCD_pixel_size'] * (np.arange(len(lsf)) - np.argmax(lsf))
    psf = lsf2psf(lsf, x, control)

    esf_vec = np.array([control['CCD_pixel_size'] * (np.arange(len(esf)) - np.argmax(np.abs(lsf))), esf]).transpose()
    lsf_vec = np.array([control['CCD_pixel_size'] * (np.arange(len(lsf)) - np.argmax(np.abs(lsf))), lsf]).transpose()
    psf_vec = np.array([control['CCD_pixel_size'] * np.arange(len(psf)), psf]).transpose()

    mtf_vec = psf2mtf(psf_vec[:, 0], psf_vec[:, 1], control)
    dqe_vec = np.array([mtf_vec[:-control['blurKernelSizeDQE'], 0], mtf2dqe(psf_vec[:, 1], mtf_vec[:, 1], control)]).transpose()

    return data, esf_vec, lsf_vec, psf_vec, mtf_vec, dqe_vec

def plot_image_characteristics(esf, lsf, psf, mtf, dqe, control):
    fig, ax = plt.subplots(1, 4)
    # Plot the three lines on the same axes
    ax[0].plot(esf[:, 0], esf[:, 1], label='esf')
    ax[1].plot(psf[:, 0], psf[:, 1], label='psf')
    ax[2].plot(mtf[:, 0], mtf[:, 1], label='mtf')
    ax[2].axhline(3, color='r')
    ax[3].plot(dqe[:, 0], dqe[:, 1], label='dqe')
    ax[2].set_yscale('log')
    ax[3].set_yscale('log')
    ax[0].legend()
    ax[1].legend()
    ax[2].legend()
    ax[3].legend()
    plt.show()

def find_closest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    
    return idx

def get_resolution(mtf, threshold=0.1):
    return mtf[find_closest(mtf[:,1], threshold),0]

def mtf2dqe2(mtf, xray_absorption):
    T2 = np.abs(mtf / 100)**2
    g1 = xray_absorption
    
    # Conversion from xray to electrons, similar to all structures
    g2 = 1.

    # For simplicity we assume the all secondary electrons are converted to light 
    g3 = 1.
    
    # The scintillation yield in hybrid structures is fixed
    scintillation_yield = 9 # 1/keV
    electron_energy = 22 # keV
    g4 = electron_energy * scintillation_yield

    dqe = 1 / (1 + (1-g1)/g1 + 1/(g1*g2) + (1-g3)/(g1*g2*g3) + 1/(g1*g2*g3*g4) + (1-T2)/(g1*g2*g3*g4*T2))
    dqe = 100 * dqe
    return dqe

"""
Here start the main module of the script
First, we define the control, which contains some general parameters
Then we extract the image characteristics
"""

control = {
           'CCD_pixel_size': 13e-3,  # in [mm]
           'blurKernelSize': 15,
           'blurKernelSizePSF': 15,
           'blurKernelSizeMTF': 20,
           'blurKernelSizeDQE': 2,
           'spare': 1,
           'nr': 200,
           }

# plt.ion()
# a = 1
# c = 1
# x = np.linspace(-10, 10, 200)
# esf_signal = a / (1 + c * np.exp(-x))
# data, esf, lsf, psf, mtf, dqe = compute_image_characteristics(esf_signal, control)
# plot_image_characteristics(esf, lsf, psf, mtf, dqe, control)
# print()

In [3]:
os.getcwd()


'C:\\Users\\orrbeer\\OneDrive - Technion\\My files\\Shared lab data\\Group members DATA\\Orr\\GitHub\\Resolution_analysis'

# Reference signal substraction
## Image is the reference - signal

In [25]:
plt.close()

tck = 'hybrid'
if os.getcwd().split('\\')[-1] == 'Analysis notebook':
    os.chdir('../')
folder = '31-28.10 backround removing/Mask-100um/'
# folder = '31-28.10 backround removing/Different_stopping/'+tck
# folder = '05.12.2023 - Cu source/'
os.chdir(folder+tck)

s = 'sig.asc' # mask-100um is '.asc' and mask-400um is '.txt'
r = 'ref.asc'
with open(s, 'r') as the_file:
    for line in the_file:
        if line.startswith('564.67523'):
            break
    for each_line in the_file:    
        N = np.array([each_line.split(",") for each_line in the_file])
sig = N[:,1:-1]
sig = np.asfarray(sig)
with open(r, 'r') as the_file:
    for line in the_file:
        if line.startswith('564.67523'):
            break
    for each_line in the_file:    
        N = np.array([each_line.split(",") for each_line in the_file])
ref = N[:,1:-1]
ref = np.asfarray(ref)
im = np.true_divide(sig,ref)
# sig[sig > 5*np.std(sig)] = np.std(sig)
image = np.rot90(im, k=-1) #dark side should be to the left, if not change k switch 1 to -1
if tck == '3.6_um':
    image[852,429] = 0.5
plt.imshow(image)#, cmap='gray')
plt.title('Sample: uniform {}um'.format(tck))
plt.savefig(tck+'.png')
plt.show()
os.chdir('../../../')


# Image aligning and slicing
## binary
is a binary mask to difrenciate between the covered and the uncovered area
## rotated
is the binary mask where the line between the cover and uncover is rotated to aligned with y axis
## roi
is the ROI where step function will be fitted to avoid overfitting and picture artefacts

In [26]:
plt.close()
def first_nonzero(arr, axis, invalid_val=-1): #Find first non-zero value in every column of a 2d array
    mask = arr>1e-6
    return np.where(mask.any(axis=axis), mask.argmax(axis=axis), invalid_val)
def line(x, slope, intercept):
    return slope*x + intercept
plt.subplot(221)
plt.imshow(image, cmap='gray')
binary = np.zeros(np.shape(image))
binary[image>2.5*np.std(image)] = 255 # if doesn't work use 1.5 instead of 2.5
binary = scipy.ndimage.median_filter(binary,size=(25,25))
plt.subplot(222)
plt.imshow(binary, cmap='gray')
vec = first_nonzero(binary[:,100:-100], axis=1, invalid_val=-1)
x = np.linspace(0,len(vec)-1,len(vec))
gmodel = Model(line)
result = gmodel.fit(vec, x=x, slope=((vec[-1]-vec[1])/(x[-1]-x[1])), intercept=vec[1])
theta = np.arctan(result.params['slope'].value)*180/np.pi
plt.subplot(223)
rotated = scipy.ndimage.rotate(binary, -theta)
plt.imshow(rotated, cmap='gray')
mid = int(np.average(first_nonzero(rotated[50:-50,:], axis=1, invalid_val=-1)))+20# Add 115 to sample 7.5, Add 10 to Hybrid, add 15 to all 400um 
roi = rotated[100:-100,mid-100:mid+100]
plt.subplot(224)
plt.imshow(roi, cmap='gray')
plt.show()
print(theta)

5.054136562464148


# Fitting 

In [27]:
plt.close()
watch=False
image2 = scipy.ndimage.rotate(image, -theta)
image2 = image2[100:-100,mid-165:mid+165]
image3 = np.zeros((np.shape(image2)[0], np.shape(image2)[1]-130)) # This is the image where the lines aligned according to the center of the st
n = np.shape(image2)[0]
slope = np.zeros(n)
p2p = np.zeros(n)
pixel = np.linspace(0,n-1,n)
for i in range(n):
    y = image2[i,:]
    x = np.linspace(0,len(y)-1,len(y))
    gmodel = Model(logistic)
    result = gmodel.fit(y, x=x, a=0.5, c = .1, d=100, b=0.2)
    slope[i] = result.params['c'].value
    p2p[i] = result.params['a'].value
    # print('i = {}, c={:.3f}'.format(i,result.params['c'].value,np.std(y)))
    if watch:
#         if n%30==0:
        plt.plot(x, y,alpha=0.5)
        plt.plot(x, result.best_fit, label='best fit')
        plt.show(block=False)
        plt.pause(0.05)
        plt.close()
#         break
    cen = int(result.params['d'].value)
    image3[i,:] = y[cen-100:cen+100]


# Summing over pixels

In [29]:
print(os.getcwd())
plt.subplot(121)
plt.imshow(image3, aspect='auto')
plt.subplot(122)
slices = 20
image4 = np.zeros((slices, np.shape(image3)[1]))
image4_stdev = np.zeros((slices, np.shape(image3)[1]))
for i in range(slices):
    one = int(len(image3[:,i])/slices)*i
    two = int(len(image3[:,i])/slices)*(i+1)
    for j in range(np.shape(image3[one:two])[1]):
        image4[i,j] = sum(image3[one:two,j])
        image4_stdev[i,j] = np.std(image3[one:two,j])
plt.imshow(image4, aspect='auto')
# plt.tight_layout()
# # slope[slope>(np.average(slope)+3.5*np.std(slope))]=np.average(slope)
# plt.savefig(folder + '/roi'+tck+'av_over50.png')
plt.show()

C:\Users\orrbeer\OneDrive - Technion\My files\Shared lab data\Group members DATA\Orr\Layers structure\x-ray measurements\EJ296


# Fitting over summed figure

In [28]:

def erfunc(x, mFL, a, b, c):
    return mFL*scipy.special.erf((x-a)/(b*np.sqrt(2)))+c
watch = True
n2 = np.shape(image4)[0]
resolution2 = np.zeros(n2)
slope2 = np.zeros(n2)
slope2std = np.zeros(n2)
p2p2 = np.zeros(n2)
pixel2 = np.linspace(0,n2-1,n2)
watch=False
control = {
               'CCD_pixel_size': 13e-3,  # in [mm]
               'blurKernelSize': 15,
               'blurKernelSizePSF': 1,
               'blurKernelSizeMTF': 20,
               'blurKernelSizeDQE': 2,
               'spare': 1,
               'nr': n2,
               }
for i in range(n2):
    y = image4[i,:]
    x = np.linspace(0,len(y)-1,len(y))
    gmodel = Model(logistic)
#     result = gmodel.fit(y, x=x, mFL=10, a=100, b = 5, c=30)
    gmodel = Model(logistic)
    result = gmodel.fit(y, x=x, a=0.5, c = .1, d=100, b=0.2)
    slope2[i] = abs(result.params['c'].value)
    slope2std[i] =result.params['c'].stderr
    p2p2[i] = abs(result.params['a'].value)
#     print('i = {}, c={:.3f}'.format(i,result.params['c'].value,np.std(y)))
    if watch:
        plt.plot(x, y,alpha=0.5)
        plt.plot(x, result.best_fit, label='best fit')
        plt.show(block=False)
        plt.pause(0.05)
        plt.close()
#     esf_signal = y # [i+400] # 
#     data, esf, lsf, psf, mtf, dqe = compute_image_characteristics(esf_signal, control)
#     resolution2[i] = get_resolution(mtf)
#     print(result.fit_report())
# fig, ax1 = plt.subplots()

sq_size = 100
center =int(11*np.shape(image3)[0]/slices)
amp = ref[mid-sq_size:mid+sq_size, center-sq_size:center+sq_size]
resolution = resolution2[np.argmax(slope2)]
plt.figure()
print(('amp; amp_error; slope; slop_error={:.0f}\t{:.0f}\t{:.3f}\t{:.3f}'.format
          (np.average(amp), np.std(amp),slope2[np.argmin(p2p2)],slope2std[np.argmin(p2p2)])))
plt.subplot(311)
plt.title('Resolution @ contrast peak={:.0f} \nAmp={:.0f}+/-{:.0f} \nslope={:.3f}+/-{:.3f}'.format
          (resolution, np.average(amp), np.std(amp),slope2[np.argmin(p2p2)],slope2std[np.argmin(p2p2)]))#,np.average(resolution),np.std(resolution)))

plt.plot(pixel2, p2p2, '--or', alpha=0.7)
plt.xlabel('Pixel')
plt.ylabel('Contrast', color='r')
plt.subplot(312)
plt.plot(pixel2, slope2, '--og', alpha=0.7)
plt.xlabel('Pixel')
plt.ylabel('Slope', color='g')
plt.subplot(313)
plt.ylabel('Resolution', color='k')
plt.xlabel('Pixel')
plt.plot(pixel2, resolution2, '--ok', alpha=0.7)
# plt.savefig(folder+'/fitting_params_'+tck+'av_over50.png')


amp; amp_error; slope; slop_error=149	11	0.446	0.031


In [30]:
plt.subplot(311)
plt.title('Resolution @ contrast peak={:.0f} \nAmp={:.0f}+/-{:.0f} \slope={:.3f}+/-{:.3f}'.format
          (resolution, np.average(amp), np.std(amp),np.min(slope2),slope2std[np.argmax(slope2)]))#,np.average(resolution),np.std(resolution)))
plt.plot(pixel2, p2p2, '--or', alpha=0.7)
plt.xlabel('Pixel')
plt.ylabel('Contrast', color='r')
plt.subplot(312)
plt.plot(pixel2, slope2, '--og', alpha=0.7)
plt.xlabel('Pixel')
plt.ylabel('Slope', color='g')
plt.subplot(313)
plt.ylabel('Resolution', color='k')
plt.xlabel('Pixel')
plt.plot(pixel2, resolution2, '--ok', alpha=0.7)
# plt.savefig(folder+'/fitting_params_'+tck+'av_over50.png')


In [32]:
plt.close()
y = image4[np.argmax(slope2),:]
y_err = image4_stdev[np.argmax(slope2),:]
y_err = y_err*20
x = np.linspace(0,len(y)-1,len(y))
gmodel = Model(logistic)
result = gmodel.fit(y, x=x, a=0.5, c = .1, d=100, b=0.2)
# slope2[i] = abs(result.params['c'].value)
# slope2std[i] =result.params['c'].stderr
# p2p2[i] = abs(result.params['a'].value)
#     print('i = {}, c={:.3f}'.format(i,result.params['c'].value,np.std(y)))
# if watch:
#         if n%30==0:
# y_err = savgol_filter(y_err,5,3)
# plt.plot(x, result.best_fit,'w--', label='best fit',linewidth=2)
plt.plot(x, y)
plt.fill_between(x, result.best_fit-y_err, result.best_fit+y_err, color='gray', alpha=0.3)
# plt.axis([0,200,18,41])
# plt.legend(['data','fit'])
plt.xlabel('Pixel')
plt.ylabel('Counts')
# plt.savefig('slope.svg',transparent=True)
plt.show()

In [33]:
# np.savetxt('6_err.txt', y_err)

In [34]:
os.getcwd()

'C:\\Users\\orrbeer\\OneDrive - Technion\\My files\\Shared lab data\\Group members DATA\\Orr\\Layers structure\\x-ray measurements\\EJ296'

In [35]:
# np.savetxt('some_BS_name.txt', ([x,y]))

In [36]:
plt.imshow(ref)
sq_size = 200

plt.plot(mid,center,'ro')
plt.plot([mid-sq_size,mid-sq_size],[center-sq_size,center+sq_size],'r--')
plt.plot([mid+sq_size,mid+sq_size],[center-sq_size,center+sq_size],'r--')
plt.plot([mid-sq_size,mid+sq_size],[center+sq_size,center+sq_size],'r--')
plt.plot([mid-sq_size,mid+sq_size],[center-sq_size,center-sq_size],'r--')
plt.figure()
plt.imshow(ref[center-sq_size:center+sq_size,mid-sq_size:mid+sq_size])
# plt.plot(mid,center,'ro')
plt.show()

# Figure for paper

In [37]:
plt.close()
bottom, top, left, right = (530, 670, 505, 605)
tck = ['hybrid','6.2','10']
for i in range(3):
    if os.getcwd().split('\\')[-1] == 'Analysis notebook':
        os.chdir('../')
    folder = '31-28.10 backround removing/Mask-100um/'
    os.chdir(folder+tck[i])
    data = ['sig.asc', 'ref.asc']
    for j in range(2):
        with open(data[j], 'r') as the_file:
            for line in the_file:
                if line.startswith('564.67523'):
                    break
            for each_line in the_file:    
                N = np.array([each_line.split(",") for each_line in the_file])
        
        if j==0:
            sig = N[:,1:-1]
            sig = np.asfarray(sig)
        if j==1:
            ref = N[:,1:-1]
            ref = np.asfarray(ref)
    os.chdir('../../../')
    os.getcwd()
    im = np.true_divide(sig,ref)
    image = np.rot90(im, k=-1) #dark side should be to the left, if not change k switch 1 to -1
#     image=scipy.ndimage.gaussian_filter(image, 0.7)#     image=scipy.ndimage.median_filter(image, size=(2,2))
    plt.subplot(1,3,i+1)
    plt.tick_params(left = False, right = False , labelleft = False, 
                labelbottom = False, bottom = False)
    plt.title(tck[i])
    if i == 0:
        image = scipy.ndimage.rotate(image, -5)
        plt.imshow(image[bottom:top,left+25:right+25],cmap='gray')#, vmin=0.345, vmax=1)

    if i == 1:
        plt.imshow(image[bottom:top,left:right],cmap='gray')#, vmin=0.345, vmax=1)
    if i == 2:
        plt.imshow(image[bottom:top,left-65:right-65],cmap='gray')#, vmin=0.345, vmax=1)
        
        pixelsize = 0.9/1024 #mm The size of a full figure is 1024x1024 pixels and measured to 0.9mm
        scalebar_size = 0.03 #mm
        scalebar_len = int(scalebar_size/pixelsize)
        ax = plt.gca()
        rect = Rectangle((60,130), scalebar_len,5, linewidth=1,edgecolor='k',facecolor='k',alpha=0.2)        
        ax.add_patch(rect)
#     plt.colorbar()
plt.show()

In [39]:
scalebar_len

34

In [38]:

# os.chdir('../')
os.getcwd()
print(left)
plt.close()

505


In [40]:
plt.close()
 
fig, ax1 = plt.subplots(figsize=(5,4.2))

left, bottom, width, height = [0.34, 0.29, 0.52, 0.56]
ax2 = fig.add_axes([left, bottom, width, height])

# ax1.plot(range(10), color='red')
ax2.plot(range(6)[::-1], color='green')

plt.show()
files = ['hyb_ESF_20_slicemax_mask_100_um','10_um_ESF_20_slicemax_mask_100_um','6.2_um_ESF_20_slicemax_mask_100_um']
for i in range(3):
    data = np.loadtxt(files[i]+'.txt')
    if i == 0:
        lbl = 'Hybrid'
        clr = 'black'
        y0 = 20.37
        x0 = 0.417
    elif i ==1:
        lbl = '10 um \nuniform'
        clr = 'blue'
        y0=18.2#18.7
        x0 = 0.626
    elif i ==2:
        lbl = '6.2 um \nuniform'
        clr = 'green'
        y0 = 19.2
        x0 = 0.480
    x = (data[0]-x0)
    y = (data[1])-y0
    xaxis = [50,150]
    gmodel = Model(logistic)
    result = gmodel.fit(y[(x>xaxis[0]) & (x<xaxis[1])], x=x[(x>xaxis[0]) & (x<xaxis[1])], a=5, c = .1, d=100, b=0.2)
#     result = gmodel.fit(y[(x>79) & (x<121)], x=x[(x>79) & (x<121)], mFL=10, a=100, b = 5, c=0)
#     gmodel = Model(logistic)
    print(result.params['b'].value)
    ax2.plot(x[(x>xaxis[0]) & (x<xaxis[1])], y[(x>xaxis[0]) & (x<xaxis[1])],'.',markersize=5, color = clr, label=lbl)
    ax2.plot(x[(x>xaxis[0]) & (x<xaxis[1])], result.best_fit,'--',linewidth=1,color = clr)#, label='best fit')
    ax2.set_xlim([92,115])
    ax2.set_ylim([0,18])
    ax2.legend()
#     print(result.fit_report())
ax2.set_xlabel('Pixel')
ax2.set_ylabel('Counts')
    

0.7455356085390913
0.7215262989368019
0.7744126443100439


Text(0, 0.5, 'Counts')

In [41]:
### Delete?

plt.close()
 
fig, ax1 = plt.subplots(figsize=(5,4.2))

left, bottom, width, height = [0.34, 0.31, 0.52, 0.56]
ax2 = fig.add_axes([left, bottom, width, height])

# ax1.plot(range(10), color='red')
# ax2.plot(range(6)[::-1], color='green')

plt.show()
files = ['hyb_ESF_20_slicemax_mask_100_um','10_um_ESF_20_slicemax_mask_100_um','6.2_um_ESF_20_slicemax_mask_100_um']
for i in range(3):
    data = np.loadtxt(files[i]+'.txt')
    if i == 0:
        lbl = 'Hybrid'
        clr = 'black'
        y0 = 20.37
        x0 = 0.417
    elif i == 1:
        lbl = '10 um \nuniform'
        clr = 'blue'
        y0 = 18.2#18.7
        x0 = 0.626
    elif i == 2:
        lbl = '6.2 um \nuniform'
        clr = 'green'
        y0 = 19.2
        x0 = 0.480
    x = ((data[0]-x0)-92)*0.9e3/1024
    y = (data[1])-y0
#     ax2.plot(x,y,'.')
    xaxis = np.array([50-92,150-92])*0.9e3/1024
    gmodel = Model(logistic)
    result = gmodel.fit(y[(x>xaxis[0]) & (x<xaxis[1])], x=x[(x>xaxis[0]) & (x<xaxis[1])], a=5, c = .1, d=6, b=0.2)
# #     result = gmodel.fit(y[(x>79) & (x<121)], x=x[(x>79) & (x<121)], mFL=10, a=100, b = 5, c=0)
# #     gmodel = Model(logistic)
#     print(result.params['b'].value)
    ax2.plot(x[(x>xaxis[0]) & (x<xaxis[1])], y[(x>xaxis[0]) & (x<xaxis[1])],'.',markersize=5, color = clr, label=lbl)
    ax2.plot(x[(x>xaxis[0]) & (x<xaxis[1])], result.best_fit,'--',linewidth=1,color = clr)#, label='best fit')
    ax2.set_xlim([0,20])
    ax2.set_ylim([0,18])
    
    ax2.legend()
#     print(result.fit_report())
ax2.set_xlabel('Distance ($\mu$m)')
ax2.set_ylabel('Counts')

Text(0, 0.5, 'Counts')

In [42]:
### Delete?

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)
files = ['hyb_ESF_20_slicemax_mask_100_um','10_ESF_6_slice10_mask_100_um','6.2_um_ESF_20_slicemax_mask_100_um']
for i in range(3):
    data = np.loadtxt(files[i]+'.txt')
    if i == 0:
        lbl = 'Hybrid'
        clr = 'black'
        y0 = 20.37
        x0 = 0.417
    elif i ==1:
        lbl = '10 um \nuniform'
        clr = 'blue'
        y0=18.5
        x0 = 0.6266
    elif i ==2:
        lbl = '6.2 um \nuniform'
        clr = 'green'
        y0 = 19.28
        x0 = 0.480
    x = data[0]
    y = (data[1])
    control = {
           'CCD_pixel_size': 13e-3,  # in [mm]
           'blurKernelSize': 1,
           'blurKernelSizePSF': 1,
           'blurKernelSizeMTF': 1,
           'blurKernelSizeDQE': 1,
           'spare': 1,
           'nr': 200,
           }
    data = np.loadtxt(files[i]+'.txt')
    y = savgol_filter(data[1],15,6)
    data, esf, lsf, psf, mtf, dqe = compute_image_characteristics(y, control)
    ax1.plot(mtf[:,0],mtf[:,1]/max(mtf[:,1]),color = clr, alpha=0.7, label=lbl)
    ax1.set_ylim([0,1.05])
    ax1.set_xlim([0,200])
    ax1.set_xlabel('Lines/pixel')
    ax1.set_ylabel('Norm MTF')
    ax1.set_xticks([0, 50, 100, 150, 200])
    print(i)
# plt.tight_layout()
plt.show()


0
1
2


C:\Users\orrbeer\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in sqrt
C:\Users\orrbeer\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide


In [43]:
# Move to another file
# plt.close()
plt.figure()
from lmfit.models import GaussianModel
import PIL
mFL = [8.81, 8.18, 7.75]
a   = [100.01, 100.04, 100.02]
b   = [3.81, 4.86, 3.44] 
c   = [8.83, 8.23, 7.77]
zeva = ['k','b','g']

## This block prove that b = sigma of 
# x = np.linspace(75,125,200)
# for i in range(3):
#     plt.plot(x, erfunc(x,mFL[i],a[i],b[i],c[i]),color=zeva[i])
#     y = np.gradient(erfunc(x,mFL[i],a[i],b[i],c[i]))/np.gradient(x)
#     plt.plot(x,y,'--',color=zeva[i])
#     mod = GaussianModel()
#     pars = mod.guess(y, x=x)
#     out = mod.fit(y, pars, x=x)
#     plt.plot(x,out.best_fit,color=zeva[i])
#     print(out.fit_report(min_correl=0.25))
import matplotlib.image
logo = np.zeros((1024,1024))
logo = plt.imread('Analysis notebook/Technion_logo.png')
y=20
x=20

plt.subplot(132)
sigma=3.81*4
logo1 =  scipy.ndimage.gaussian_filter(logo, (sigma,sigma,-1))
plt.imshow(logo1)#, cmap='gray')
plt.tick_params(left = False, right = False , labelleft = False, 
                labelbottom = False, bottom = False)
plt.title('Imaging with hybrid scintillator')
plt.subplot(133)
sigma=4.86*4
logo2 =  scipy.ndimage.gaussian_filter(logo, (sigma,sigma,0))
plt.imshow(logo2)#, cmap='gray')
plt.tick_params(left = False, right = False , labelleft = False, 
                labelbottom = False, bottom = False)
plt.title('Imaging with uniform scintillator\nwith equivalent intensity')
logo[1850:1900,50:250]=[0, 0, 0, 1]
plt.subplot(131)
plt.imshow(logo)#, cmap='gray')
# plt.axis([512-x,512+x,512-y,512+y])
plt.tick_params(left = False, right = False , labelleft = False, 
                labelbottom = False, bottom = False)
plt.title('Original image')

Text(0.5, 1.0, 'Original image')

In [44]:
os.getcwd()
os.chdir(r'C:\Users\orrbeer\OneDrive - Technion\My files\Shared lab data\Group members DATA\Orr\Layers structure\x-ray measurements\EJ296')

In [45]:
print ('Pixel size = {} m'.format(0.5e-3/1024*200))

Pixel size = 9.765625e-05 m


# Silverfish

In [46]:
from matplotlib.patches import Rectangle
plt.figure(figsize=(8,3))
os.chdir('15.01.2023 - silverfish')
im = ['hyb200','uni6_2um']
for i in range(2):
    image = np.loadtxt('silverfish_'+im[i])
    plt.subplot(1,2,i+1)
    
    plt.gca().axes.get_xaxis().set_visible(False)
    plt.gca().axes.get_yaxis().set_visible(False)
    if i == 0:
        plt.title('6 $\mu m$ heterostructure scintillator')
        image = image-.48
    if i == 1:
        image = image-.98
        plt.title('6.2 $\mu m$ uniform scintillator')
        image=np.rot90(image,k=2)
#         plt.axis([144,494,58,425])
        pixelsize = 2.54 #um
        scalebar_size = 500 #um
        scalebar_len = int(scalebar_size/pixelsize)
        ax = plt.gca()
        rect = Rectangle((800,920), scalebar_len, 50, linewidth=1,edgecolor='k',facecolor='k',alpha=0.1)
    if i == 2:
        plt.title('6.2 $\mu m$ uniform scintillator')
        image=np.rot90(image,k=2)
    plt.imshow(image, cmap='gray')#, vmin =0.3 , vmax = 1.2)
    

# Add the patch to the Axes
    ax.add_patch(rect)
    plt.matplotlib.pyplot.colorbar()
    plt.tight_layout()
os.chdir('../')

In [47]:
combined_data = np.array([a, b, c])
#Get the min and max of all your data
_min, _max = np.amin(combined_data), np.amax(combined_data)
a

[100.01, 100.04, 100.02]

In [48]:
# os.chdir('15.01.2023 - silverfish')
os.chdir('15.01.2023 - silverfish')
# if os.getcwd().split('\\')[-1] == 'Analysis notebook':
#     os.chdir('../')
# folder = '15.01.2023 - silverfish'
# os.chdir(folder)
data = 'silverfishvis.asc'
with open(data, 'r') as the_file:
    for line in the_file:
        if line.startswith('564.67523'):
            break
    for each_line in the_file:    
        N = np.array([each_line.split(",") for each_line in the_file])
    im = np.asfarray(N[:,1:-1])
os.chdir('../')

In [49]:
plt.close()
im=np.rot90(im,k=1)
plt.imshow(im, cmap='gray')
plt.show()

# Adding DQE

In [50]:
## X-ray absorbption ##
Ti_mu_over_rho_20keV = 1.585E+01
O_mu_over_rho_20keV  = 8.651E-01
Cl_mu_over_rho_20keV = 7.739E+00
C_mu_over_rho_20keV  = 4.420E-01
H_mu_over_rho_20keV  = 3.695E-01

# Ti_mu_over_rho_10keV = 1.107E+02
# O_mu_over_rho_10keV  = 5.952E+00
# Cl_mu_over_rho_10keV = 5.725E+01
# C_mu_over_rho_10keV  = 2.373E+00
# H_mu_over_rho_10keV  = 3.854E-01

# TiOx in PVAl wt% in the 30wt% samples
Ti_wt = 31.4/100
O_wt  = 36.6/100
Cl_wt = 6.0 /100
C_wt  = 26.0/100
# mu_over_rho \Sigma_i w_i * mu_over_rho_i
mu_over_rho_TiOxPVA = (Ti_wt*Ti_mu_over_rho_20keV + O_wt*O_mu_over_rho_20keV +
                       Cl_wt*Cl_mu_over_rho_20keV + C_wt*C_mu_over_rho_20keV)
rho_TiOxPVA = 1.95
# PVT wt fraction (C9H10)
C_at_mass = 12
H_at_mass = 1
C_wt =  9/19*C_at_mass/(9/19*C_at_mass+10/19*H_at_mass)
H_wt = 10/19*H_at_mass/(9/19*C_at_mass+10/19*H_at_mass)
mu_over_rho_PVT = C_wt*C_mu_over_rho_20keV + H_wt*H_mu_over_rho_20keV

rho_PVT = 1.25

t_uniforms = np.array([6.2e-4, 10e-4]) # thicknesses of uniform layers in cm
rel_abs_uniforms = 1-np.exp(-t_uniforms*mu_over_rho_PVT*rho_PVT)

rho_TiOxPVA = 1.855
t_TiOxPVA = 200e-7 # thicknesses of single TiOxPVA layers in cm
t_PVT = 1000e-7 # the thickness of single layer of PVT scintillator in all of the samples.
rel_abs_hybrids = 1-np.exp(-5*(t_TiOxPVA*mu_over_rho_TiOxPVA*rho_TiOxPVA+
                    t_PVT*mu_over_rho_PVT*rho_PVT))
print("Uniform layer x-ray absorption: {}".format(rel_abs_uniforms))
print("hybrid x-ray absorption: {}".format(rel_abs_hybrids))

Uniform layer x-ray absorption: [0.00033773 0.00054467]
hybrid x-ray absorption: 0.0013608850269171135


In [51]:
control = {
           'CCD_pixel_size': 0.0008789,  # in [mm]
           'blurKernelSize': 1,
           'blurKernelSizePSF': 1,
           'blurKernelSizeMTF': 1,
           'blurKernelSizeDQE': 1,
           'spare': 1,
           'nr': 200,
           }
plt.close()
fig = plt.figure(figsize=[8, 4])
ax1 = fig.add_subplot(141)
ax2 = fig.add_subplot(142)
ax3 = fig.add_subplot(143)
files = ['hyb_ESF_20_slicemax_mask_100_um','10_um_ESF_20_slicemax_mask_100_um','6.2_um_ESF_20_slicemax_mask_100_um']
files_err = ['hyb_err','10_err','6_err']
for i in range(3):
    data = np.loadtxt(files[i]+'.txt')
    data_err = np.loadtxt(files_err[i]+'.txt')
    if i == 0:
        lbl = 'Hetero-\nstructure'
        clr = 'black'
        y0 = 20.37
        x0 = 0.417
        x_ray_abs = 0.00136
        sl_thickness = 0.2
    elif i == 1:
        lbl = '10 um \nhomogeneous'
        clr = 'blue'
        y0 = 18.2#18.7
        x0 = 0.626
        x_ray_abs = 0.000545
        sl_thickness = 0
    elif i == 2:
        lbl = '6.2 um \nhomogeneous'
        clr = 'green'
        y0 = 19.2
        x0 = 0.480
        x_ray_abs = 0.000338
        sl_thickness = 0
        
    x = ((data[0]-x0)-92)*0.9e3/1024
    y = (data[1])-y0
    y_err = data_err
    xaxis = np.array([50-92,150-92])*0.9e3/1024
    gmodel = Model(logistic)
    result = gmodel.fit(y[(x>xaxis[0]) & (x<xaxis[1])], x=x[(x>xaxis[0]) & (x<xaxis[1])], a=5, c = .1, d=6, b=0.2)
# #     result = gmodel.fit(y[(x>79) & (x<121)], x=x[(x>79) & (x<121)], mFL=10, a=100, b = 5, c=0)
# #     gmodel = Model(logistic)
#     print(result.params['b'].value)
    
    ax1.plot(x[(x>xaxis[0]) & (x<xaxis[1])], y[(x>xaxis[0]) & (x<xaxis[1])]
             ,'.', markersize=5 , color = clr, label=lbl, alpha = 0.8)
    ax1.plot(x[(x>xaxis[0]) & (x<xaxis[1])], result.best_fit,'--',linewidth=1,color = clr)#, label='best fit')
    ax1.fill_between(x[(x>xaxis[0]) & (x<xaxis[1])], result.best_fit-y_err[(x>xaxis[0]) & (x<xaxis[1])],
                     result.best_fit+y_err[(x>xaxis[0]) & (x<xaxis[1])], color=clr, alpha=0.1)
    ax1.set_xlim([0,20])
    ax1.set_ylim([0,18])
    ax1.set_xlabel('Distance ($\mu$m)')
    ax1.set_ylabel('Counts (a.u.)')
    
    
    
    x = data[0]
    y = (data[1])
    data = np.loadtxt(files[i]+'.txt')
    y = savgol_filter(data[1],15,6)
    dy =  abs(np.gradient(y[:-6],x[1]-x[0]))
    data, esf, lsf, psf, mtf, dqe = compute_image_characteristics(y, control)
    x = mtf[:,0] # The units of mtf[:,0] is lines/pixels. The value 1024/0.9e3 is pixels per um
    dx = x[1]-x[0]
    norm_mtf = mtf[:,1]/max(mtf[:,1])
#     norm_mtf = savgol_filter(norm_mtf,5,1)
    d_mtf =  abs(np.gradient(norm_mtf))
    dy_mtf =  abs(d_mtf/dy)
    err_mtf = abs(dy_mtf*y_err[6:])
#     ax2.plot(x,norm_mtf,color = clr, alpha=0.7, label=lbl)
#     ax2.fill_between(x, norm_mtf-err_mtf,norm_mtf+err_mtf, color=clr, alpha=0.1)
    ax2.plot(x, savgol_filter(err_mtf,15,1), color=clr, alpha=0.5)
    ax2.set_ylim([0,1.4])
    ax2.set_xlim([0,300])
    ax2.set_xlabel('Lines/mm')
    ax2.set_ylabel('MTF (Norm.)')
    ax2.legend(frameon=False) # 
    
    
    dqe = mtf2dqe2(mtf[:,1], x_ray_abs,sl_thickness,blurKernelSizeDQE=1)
    ax3.plot(x,savgol_filter(dqe,9,1),color = clr, alpha=0.7, label=lbl)
    ax3.set_xlabel('Lines/mm')
    ax3.set_ylabel('DQE')
    ax3.set_xlim([0,300])
# The units of mtf[:,0] is lines/pixels. The value 1024/0.9e3 is pixels per um
ETF = [732, 507, 357, 263]
ETF_err = [13, 22, 35, 69]
intensity = [93, 118, 146, 188]
intensity_err = [11 ,11 ,12 ,12]
ETFh = 481
ETFh_err = 30
inth = 149
inth_err =11
ax4 = fig.add_subplot(144)
ax4.errorbar(x=ETF, y=intensity, xerr=ETF_err, yerr=intensity_err, linestyle='',fmt='.',markersize=10, 
             markeredgecolor='tab:blue', markerfacecolor=(0, 0.1, 0.2, 0.3))
ax4.errorbar(x=ETFh, y=inth, xerr=ETFh_err, yerr=inth_err,fmt='.',markersize=10, 
             markeredgecolor='tab:orange', markerfacecolor=(0.3, 0.1, 0.1, 0.3))

x=np.linspace(150,800,200)
y = 8246.5*x**(-0.683)
ax4.plot(x,y,'--',color='tab:blue', alpha=0.8)
ax4.set_xlim([150,800])
ax4.set_ylabel('Intensity (a.u.)')
ax4.set_xlabel('ESF slope (mm$^{-1}$)')
# secx = ax4.secondary_xaxis('top')
# secx.set_xticks([728.2, 512, 364.1, 318.6])
# secx.set_xticklabels([3.8, 6.2, 10, 15], fontsize=10)#, rotation=10)
# secx.set_xlabel('Volume of\n stopping layer (%)', fontsize=14)
# plt.tight_layout()

plt.show()

C:\Users\orrbeer\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in sqrt
C:\Users\orrbeer\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


TypeError: mtf2dqe2() got an unexpected keyword argument 'blurKernelSizeDQE'

In [ ]:
os.getcwd()
# plt.close()
# order = 1
# window = 15
# plt.plot(savgol_filter(y_err,window, order),label='yerr')
# plt.plot(savgol_filter(dy,window, order),label='dy')
# plt.plot(savgol_filter(d_mtf,window, order),label='dmtf')
# plt.plot(savgol_filter(d_mtf,window, order)*savgol_filter(y_err[0:-6],window, order)/savgol_filter(dy,window, order))
len(x)

In [ ]:
lbl = ['Hybrid','10 um \nuniform', '6.2 um \nuniform']
clr = ['black', 'blue', 'green']
fig = plt.figure()
ax2 = fig.add_subplot(111)
ax2.plot([0,300],[0.1,0.1],'m--',label='10% MTF')
for i in range(3):
    data = np.loadtxt(files[i]+'.txt')
    
    x = data[0]
    y = (data[1])
    data = np.loadtxt(files[i]+'.txt')
    y = savgol_filter(data[1],15,6)
    data, esf, lsf, psf, mtf, dqe = compute_image_characteristics(y, control)
    x = mtf[:,0] # The units of mtf[:,0] is lines/pixels. The value 1024/0.9e3 is pixels per um
    ax2.plot(x,mtf[:,1]/max(mtf[:,1]),color = clr[i], alpha=0.7, label=lbl[i])
    ax2.set_ylim([0,1.05])
    ax2.set_xlim([0,300])
    ax2.set_xlabel('Lines/mm')
    ax2.set_ylabel('MTF (Norm.)')
    ax2.legend(frameon=False) # 


In [ ]:
os.chdir('EJ296') #EJ296

In [ ]:
def mtf2dqe2(mtf, xray_absorption, sl_thickness, blurKernelSizeDQE):
    T2 = np.abs(mtf / 100)**2
    g1 = xray_absorption
    
    # Conversion from xray to electrons, similar to all structures
    g2 = 1.

    # The secondary electrons that reach the scintillator are converted to light
    electronic_absorption = 10.7 # 1/um
    N = 1001
    depths = np.linspace(0, sl_thickness, N)
    reaching_electrons = np.exp(-electronic_absorption * depths) # The amound of electrons reaching the scintillator from a each depth
    g3 = np.sum(reaching_electrons) * (1/N)
    
    # The scintillation yield in hybrid structures is fixed
    scintillation_yield = 9 # 1/keV
    electron_energy = 22 # keV
    g4 = electron_energy * scintillation_yield

    dqe = 1 / (1 + (1-g1)/g1 + 1/(g1*g2) + (1-g3)/(g1*g2*g3) + 1/(g1*g2*g3*g4) + (1-T2)/(g1*g2*g3*g4*T2))
    dqe = 100 * dqe#savgol_filter(dqe, blurKernelSizeDQE, 1)
    return dqe

In [ ]:
mtf2dqe(mtf,0.00136,3)

In [ ]:
ETF = [728.2, 512, 364.1, 318.6] 
intensity = [93, 118, 146, 188]
ETFh = 455.1
inth = 150
# plt.subplot(144)
plt.plot(ETF, intensity,'r')
plt.plot(ETFh, inth,'b')
plt.tight_layout()


In [ ]:
pfolder = '970'
# folder = '31-28.10 backround removing/Different_stopping/'+tck
# folder = '05.12.2023 - Cu source/'
os.chdir(pfolder)

# s = 'hyb_silverfish_ref.asc' # mask-100um is '.asc' and mask-400um is '.txt'
r = 'ref.asc'

with open(r, 'r') as the_file:
    for line in the_file:
        if line.startswith('564.67523'):
            break
    for each_line in the_file:    
        N = np.array([each_line.split(",") for each_line in the_file])
ref = N[:,1:-1]
ref = np.asfarray(ref)
plt.imshow(ref,cmap='gray')

In [ ]:
os.chdir('../')

In [ ]:
os.getcwd()
